In [46]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import requests
import json
import os
import pandas as pd  # nécessites les librairies  openpyxl, xlrd pour ouvrir le xlsx
import shapely
from shapely.geometry import Point, LineString, Polygon
import sys

# Bibliothèques de fonctions STEAN
from stean_plus import instanceST 

# Fichier de paramètres
import configparser # Permet de parser le fichier de paramètres

# Affichage complet des dataframes
pd.set_option("display.max_colwidth", None)

# Lecture du fichier de configuration
config = configparser.RawConfigParser() # On créé un nouvel objet "config"
config.read('param.conf') # On lit le fichier de paramètres

['param.conf']

In [ ]:
# Déclaration des variables ########################################################################### 
# Fichier Excel de configuration
path_file_conf = config['CONFIGURATION']['conf_file']

# Création d'une instance SensorThing ###########################################################################
#sessionST=instanceST('url de votre instance STEAN', 'login de votre instance STEAN', 'mot de passe de votre instance STEAN')
url_STEAN = config['SERVICE']['url']
login = config['SERVICE']['user']
password = config['SERVICE']['pw']
sessionST=instanceST(url_STEAN, login, password)

# Lecture de la configuration ###########################################################################
# liste des onglets à prendre en compte
sheet_list = ['1_observedProperty','2_sensor','3_thing','4_datastream','5_featureOfInterest']

with pd.ExcelFile(path_file_conf) as xls:
    for entity in sheet_list:
        print("Lecture de l'onglet " + entity)
        conf_table = pd.read_excel(xls, sheet_name = entity)
        # suppression des lignes vides
        if conf_table.isnull().values.any() == True: 
            print("Attention! Détection de cellules vides dans l'onglet " + entity)
            print("Suppression UNIQUEMENT des lignes entièrement vides")
            conf_table.dropna(how='all', inplace=True)
            
        # arrêt conf_table = du script si présence doublons dans les colonnes name
        if conf_table.duplicated(subset=['name']).values.any() == True:
            print("Attention! Détection de doublons dans l'onglet " + entity + " dans la colonne name")
            sys.exit("Arrêt du script! Veuillez vérifier le fichier de configuration " + path)
        # mise en mémoire des contenus de chaques onglets du fichier de configuration
        # supprression des caractères espace, \n, \r et \n
        if entity == '4_datastream': datastream_table = conf_table.map(lambda x: x.strip(' \r\n\t') if isinstance(x, str) else x)
        if entity == '1_observedProperty': observedProperty_table = conf_table.map(lambda x: x.strip(' \r\n\t') if isinstance(x, str) else x)
        if entity == '2_sensor': sensor_table = conf_table.map(lambda x: x.strip(' \r\n\t') if isinstance(x, str) else x)   
        if entity == '3_thing': thing_table = conf_table.map(lambda x: x.strip(' \r\n\t') if isinstance(x, str) else x)  
        if entity == '5_featureOfInterest': featureOfInterest_table = conf_table.map(lambda x: x.strip(' \r\n\t') if isinstance(x, str) else x)


# Vérification des coordonnées des onglets 3_thing et 5_featureOfInterest
print("Vérification des coordonnées de l'onglet 5_featureOfInterest")
for index, row in featureOfInterest_table.iterrows():
    if row['feature.type'] == "Point" and Point(json.loads(row['feature.coordinates'])).is_valid == False:
        print("Erreur format point pour", row['name'])
        sys.exit("Arrêt du script! Veuillez vérifier le fichier de configuration " + path)
    if row['feature.type'] == "Polygon" and Polygon(json.loads(row['feature.coordinates'])).is_valid == False:
        print("Erreur format Polygone pour", row['name']) 
        sys.exit("Arrêt du script! Veuillez vérifier le fichier de configuration " + path)

print("Vérification des coordonnées de l'onglet 3_thing")
for index, row in thing_table.iterrows():
    if row['type'] == "Point" and Point(json.loads(row['coordinates'])).is_valid == False:
        print("Erreur format point pour", row['name'])
        sys.exit("Arrêt du script! Veuillez vérifier le fichier de configuration " + path)
    if row['type'] == "Polygon" and Polygon(json.loads(row['coordinates'])).is_valid == False:
        print("Erreur format Polygone pour", row['name']) 
        sys.exit("Arrêt du script! Veuillez vérifier le fichier de configuration " + path)

# Création des fichiers Json (featureOfInterest, observedProperty, sensor,thing) ###########################################################################
# Ordre de création: 1. observedProperty, 2. sensor, , 3. thing et location.  
# L'entité **datastream** sera crée après envoie des 3 scripts Json ci-dessus.
# La création du datastream_json nécessite la récupération d'identifiant au préalable

featureOfInterest_json = sessionST.featureOfInterest_create(featureOfInterest_table)
observedProperty_json = sessionST.observedProperty_create(observedProperty_table)
sensor_json = sessionST.sensor_create(sensor_table)
thing_json = sessionST.thing_create(thing_table)

# Configuration des entités (FeaturesOfInterest, ObservedProperties, Sensors, Things) par envoi de Json
if len(observedProperty_json)!=0 and len(sensor_json)!=0 and len(thing_json)!=0 and len(featureOfInterest_json)!=0:
    sessionST.connexion() # connexion au serveur
    sessionST.post_data_serveur("FeaturesOfInterest",featureOfInterest_json)
    sessionST.post_data_serveur("ObservedProperties",observedProperty_json)
    sessionST.post_data_serveur("Sensors",sensor_json)
    sessionST.post_data_serveur("Things",thing_json)
    sessionST.log_out()

Lecture de l'onglet 1_observedProperty
Lecture de l'onglet 2_sensor
Attention! Détection de cellules vides dans l'onglet 2_sensor
Suppression UNIQUEMENT des lignes entièrement vides
Lecture de l'onglet 3_thing
Attention! Détection de cellules vides dans l'onglet 3_thing
Suppression UNIQUEMENT des lignes entièrement vides
Lecture de l'onglet 4_datastream
Attention! Détection de cellules vides dans l'onglet 4_datastream
Suppression UNIQUEMENT des lignes entièrement vides
Lecture de l'onglet 5_featureOfInterest
Attention! Détection de cellules vides dans l'onglet 5_featureOfInterest
Suppression UNIQUEMENT des lignes entièrement vides
Vérification des coordonnées de l'onglet 5_featureOfInterest
Vérification des coordonnées de l'onglet 3_thing

 Publication des FeaturesOfInterest
check if exist FeaturesOfInterest
Get Data
FeaturesOfInterest déjà publié : ['Outlet E30', 'Outlet Exutoire', 'Piezometer PK1', 'Piezometer PK2', 'Piezometer PK3', 'Piezometer PK4', 'Piezometer PK5', 'Piezometer PK

c:\Users\cgeneste\AppData\Local\miniconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\cgeneste\AppData\Local\miniconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
c:\Users\cgeneste\AppData\Local\miniconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [49]:
# Création du fichier Json (datastream) ###########################################################################
datastream_json = sessionST.datastream_create(datastream_table, observedProperty_table, sensor_table, thing_table, featureOfInterest_table)



 Publication des datastreams
check if exist Datastreams
Get Data
next link
next link
Datastreams déjà publié : ['Outlet E30_OTT Thalimede_stream level', 'Outlet E30_OTT PLS C_stream level', 'Outlet E30_OTT PLS C_water temperature', 'Outlet E30_OTT PLS C_electric conductiviy', 'Outlet E30_Wimesure Pt100_water temperature', 'Outlet E30_OTT Limnigraphe_stream level', 'Outlet E30_Perkin Elmer 3100 Atomic Absorption Spectrometer_potassium', 'Outlet E30_Perkin Elmer 3100 Atomic Absorption Spectrometer_magnesium', 'Outlet E30_Perkin Elmer 3100 Atomic Absorption Spectrometer_sodium', 'Outlet E30_Perkin Elmer 3100 Atomic Absorption Spectrometer_calcium', 'Outlet E30_Technicon auto analyzer_nitrate', 'Outlet E30_Technicon auto analyzer_ammonium', 'Outlet E30_Mohr Method titration_chloride', 'Outlet E30_Multiparameter probe_water temperature', 'Outlet E30_Multiparameter probe_electric conductiviy', 'Outlet E30_Multiparameter probe_pH', 'Outlet E30_Multiparameter probe_Redox potential', 'Outlet E

In [ ]:
# Configuration des entités (Datastreams) par envoi de Json
if len(datastream_json)!=0:
    sessionST.connexion() # connexion au serveur
    sessionST.post_data_serveur("Datastreams",datastream_json)
    sessionST.log_out()

Connexion agrhys  à  http://localhost:8029/agrhys/v1.1/
Connexion OK à  http://localhost:8029/agrhys/v1.1/ 

Post Data
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
Post OK
P